In [2]:
from torch.utils.data import Dataset , DataLoader
import torch
class GPTdatasetV1(Dataset):
    def __init__(self,text,tokenizer,max_length,stride):
        self.input_ids = []
        self.output_ids =[]

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            output_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(output_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx], self.output_ids[idx]

In [3]:
import tiktoken

In [4]:
def create_dataloader_v1(text,batch_size=4,max_length = 256,stride =128,shuffle = True,drop_last = True,num_workers =0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTdatasetV1(text,tokenizer,max_length, stride)
    # creating dataloader
    dataloader = DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=num_workers)
    return dataloader

In [5]:
import torch
vocab_size = 50257
output_dim = 256

token_embedding_layer  = torch.nn.Embedding(vocab_size,output_dim)

In [6]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()

In [8]:
max_length = 4
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
input1,target1 = next(data_iter)
# print(input1,target1)

In [11]:
print("Input tokens:\n",input1)
print("\ndimension:\n",input1.shape)

Input tokens:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

dimension:
 torch.Size([8, 4])


### now we need to convert these tokens to vector embedding and add postional Encoding to it

In [13]:
token_embedded = token_embedding_layer(input1)
print(token_embedded.shape)

torch.Size([8, 4, 256])


In [15]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)
pos_embedding = pos_embedding_layer(torch.arange(max_length))

In [16]:
print(pos_embedding.shape)

torch.Size([4, 256])


In [17]:
final_input = token_embedded+pos_embedding

In [19]:
print(final_input.shape)

torch.Size([8, 4, 256])


These final input is ready to be passed in the training, this is only one input from whole text and we need to do this input prepration for whole text